# 10 数据聚合与分组操作
1 使用一个或多个键将pandas对象拆分
2 计算组内汇总统计信息，如计数，平均值或标准差或用户定义的函数
3 应用组内变换或其他操作，如标准化、线性回归、排位或子集选择
4 计算数据透视表和交叉表
5 执行分位数分析和其他统计组分析

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key1': list('aabba'),
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.844591,-1.393349
1,a,two,0.555978,0.910947
2,b,one,1.354937,-0.938447
3,b,two,0.101097,0.004430
4,a,one,-0.994137,-0.911844


In [3]:
# 根据key1计算data1的平均值
df['data1'].groupby(df['key1']).mean()

key1
a   -0.427583
b    0.728017
Name: data1, dtype: float64

In [4]:
# 也可以使用多列进行groupby，传入一个列表
df['data1'].groupby([df['key1'], df['key2']]).mean()

key1  key2
a     one    -0.919364
      two     0.555978
b     one     1.354937
      two     0.101097
Name: data1, dtype: float64

In [5]:
# 改变顺序不影响最终结果
# groupby和mean操作之后发生了什么？
# 首先根据分组键进行了聚合，并产生一个新的Series，
# 这个Series使用key1，key2组合后的唯一值作为索引，然后计算data1对应索引上的均值
df.groupby([df['key1'], df['key2']])['data1'].mean()

key1  key2
a     one    -0.919364
      two     0.555978
b     one     1.354937
      two     0.101097
Name: data1, dtype: float64

In [6]:
df

,key1,key2,data1,data2
0,a,one,-0.844591,-1.393349
1,a,two,0.555978,0.910947
2,b,one,1.354937,-0.938447
3,b,two,0.101097,0.004430
4,a,one,-0.994137,-0.911844


In [8]:
# 分组键可以是正确长度的任意数组
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2006, 2005, 2006, 2005])
df.groupby([states, years])['data1'].mean()

California  2005    1.354937
            2006    0.555978
Ohio        2005   -0.919364
            2006    0.101097
Name: data1, dtype: float64

In [9]:
# 传递列名也可以作为分组键
# 可以看到结果中并没有key2，因为key2并不是数值数据
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.427583,-0.464749
b,0.728017,-0.467009


In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.919364 -1.152596
     two   0.555978  0.910947
b    one   1.354937 -0.938447
     two   0.101097  0.004430

In [13]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 遍历各分组

In [14]:
# groupby对象支持迭代，会生成一个包含组名和数据块的2维元组序列
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.844591 -1.393349
1    a  two  0.555978  0.910947
4    a  one -0.994137 -0.911844
b
  key1 key2     data1     data2
2    b  one  1.354937 -0.938447
3    b  two  0.101097  0.004430


In [15]:
# 在多个分组键的情况下，元组中的第一个元素是键值的元组
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.844591 -1.393349
4    a  one -0.994137 -0.911844
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.555978  0.910947
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.354937 -0.938447
('b', 'two')
  key1 key2     data1    data2
3    b  two  0.101097  0.00443


In [16]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.844591 -1.393349
 1    a  two  0.555978  0.910947
 4    a  one -0.994137 -0.911844,
 'b':   key1 key2     data1     data2
 2    b  one  1.354937 -0.938447
 3    b  two  0.101097  0.004430}

### 10.1.2 选择一列或所有列的子集

In [18]:
# 只有data1
df.groupby('key1')['data1'].sum()

key1
a   -1.282750
b    1.456034
Name: data1, dtype: float64

In [21]:
df['data1'].groupby(df['key1']).sum()

key1
a   -1.282750
b    1.456034
Name: data1, dtype: float64

In [22]:
# 选择一部分数据列，只需要指定就好
df.groupby(['key1', 'key2'])['data2'].sum()

key1  key2
a     one    -2.305193
      two     0.910947
b     one    -0.938447
      two     0.004430
Name: data2, dtype: float64

In [23]:
# 1 如果只有单个列名作为标量传递，则为分组的Series
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [24]:
s_grouped.mean()

key1  key2
a     one    -1.152596
      two     0.910947
b     one    -0.938447
      two     0.004430
Name: data2, dtype: float64

In [26]:
# 2 如果传递的是列表或数组，则此索引操作返回的对象是分组的DataFrame
# 传入列表，返回DataFrame
s_grouped = df.groupby('key1')[['data2']]
s_grouped

### 10.1.3 使用字典和Series分组

In [27]:
# 分组信息可能会以非数组形式存在
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=list('abcde'),
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.553660,-2.508598,0.709008,0.021766,-0.926256
Steve,0.736930,0.450232,1.831150,0.819195,0.978891
Wes,0.961692,-0.405816,-0.669264,0.104635,0.213450
Jim,-0.805714,1.622042,0.932485,-1.750847,-2.252258
Travis,-0.590660,-0.622607,1.144758,-1.018615,-0.458426


In [29]:
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.553660,-2.508598,0.709008,0.021766,-0.926256
Steve,0.736930,0.450232,1.831150,0.819195,0.978891
Wes,0.961692,NaN,NaN,0.104635,0.213450
Jim,-0.805714,1.622042,0.932485,-1.750847,-2.252258
Travis,-0.590660,-0.622607,1.144758,-1.018615,-0.458426


In [30]:
# 各列的分组对应关系，并且想把各列按组累加
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [31]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.730774,-2.881193
Steve,2.650345,2.166053
Wes,0.104635,1.175141
Jim,-0.818363,-1.435931
Travis,0.126143,-1.671693


In [32]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

### 10.1.4 使用函数分组

In [33]:
# 按照名字的长度分组
people.groupby(len).sum()

,a,b,c,d,e
3,0.709638,-0.886556,1.641493,-1.624447,-2.965064
5,0.736930,0.450232,1.831150,0.819195,0.978891
6,-0.590660,-0.622607,1.144758,-1.018615,-0.458426


In [34]:
people

,a,b,c,d,e
Joe,0.553660,-2.508598,0.709008,0.021766,-0.926256
Steve,0.736930,0.450232,1.831150,0.819195,0.978891
Wes,0.961692,NaN,NaN,0.104635,0.213450
Jim,-0.805714,1.622042,0.932485,-1.750847,-2.252258
Travis,-0.590660,-0.622607,1.144758,-1.018615,-0.458426


In [35]:
# 将函数与数组、字典或Series进行混合并不困难，所有的对象那个都会在内部转换为数组
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).sum()

a         b         c         d         e
3 one  1.515352 -2.508598  0.709008  0.126401 -0.712806
  two -0.805714  1.622042  0.932485 -1.750847 -2.252258
5 one  0.736930  0.450232  1.831150  0.819195  0.978891
6 two -0.590660 -0.622607  1.144758 -1.018615 -0.458426

### 10.1.5 根据索引层级分组

In [36]:
# 分层索引的数据有一个非常方便的地方，就是能够在周索引的某个层级上进行聚合
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      1.069803 -0.845073  2.057548 -1.672885 -0.760306
1      1.250289  1.074574  1.148241  1.187818  0.576493
2      0.350411 -0.100084  0.406645  0.983204 -0.178269
3     -0.485553 -1.253105  0.632496 -0.850636 -0.401723

In [37]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 数据聚合

In [38]:
df

,key1,key2,data1,data2
0,a,one,-0.844591,-1.393349
1,a,two,0.555978,0.910947
2,b,one,1.354937,-0.938447
3,b,two,0.101097,0.004430
4,a,one,-0.994137,-0.911844


In [41]:
df.groupby('key1')['data1'].quantile(0.5)

key1
a   -0.844591
b    0.728017
Name: data1, dtype: float64

In [43]:
# 要使用子集的聚合函数，需要将函数传递给aggregate或agg方法
def peak_to_peak(arr):
    return arr.max() - arr.min()


df.groupby('key1')['data1'].agg(peak_to_peak)

key1
a    1.550115
b    1.253841
Name: data1, dtype: float64

In [45]:
# 列出groupby默认的所有聚合结果
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.427583  0.855065 -0.994137 -0.919364 -0.844591 -0.144306   
b      2.0  0.728017  0.886599  0.101097  0.414557  0.728017  1.041477   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.555978   3.0 -0.464749  1.215469 -1.393349 -1.152596 -0.911844   
b     1.354937   2.0 -0.467009  0.666714 -0.938447 -0.702728 -0.467009   

                          
           75%       max  
key1                      
a    -0.000449  0.910947  
b    -0.231290  0.004430

### 10.2.1 逐列及多函数应用

In [46]:
tips = pd.read_csv('./examples/tips.csv')
tips['tip_pct'] = tips['tip'] / (tips['total_bill'])
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [47]:
grouped = tips.groupby(['smoker', 'day'])
grouped.agg('mean')

total_bill       tip      size   tip_pct
smoker day                                           
No     Fri    18.420000  2.812500  2.250000  0.151650
       Sat    19.661778  3.102889  2.555556  0.158048
       Sun    20.506667  3.167895  2.929825  0.160113
       Thur   17.113111  2.673778  2.488889  0.160298
Yes    Fri    16.813333  2.714000  2.066667  0.174783
       Sat    21.276667  2.875476  2.476190  0.147906
       Sun    24.120000  3.516842  2.578947  0.187250
       Thur   19.190588  3.030000  2.352941  0.163863

In [49]:
# 你可以将函数名以字符串形式传递
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

smoker  day 
No      Fri     0.151650
        Sat     0.158048
        Sun     0.160113
        Thur    0.160298
Yes     Fri     0.174783
        Sat     0.147906
        Sun     0.187250
        Thur    0.163863
Name: tip_pct, dtype: float64

In [50]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
smoker day                                   
No     Fri   0.151650  0.028123      0.067349
       Sat   0.158048  0.039767      0.235193
       Sun   0.160113  0.042347      0.193226
       Thur  0.160298  0.038774      0.193350
Yes    Fri   0.174783  0.051293      0.159925
       Sat   0.147906  0.061375      0.290095
       Sun   0.187250  0.154134      0.644685
       Thur  0.163863  0.039389      0.151240

In [52]:
# 传递的是(name, function)元组的列表，每个元组的第一个元素将作为DataFrame的列名
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
smoker day                     
No     Fri   0.151650  0.028123
       Sat   0.158048  0.039767
       Sun   0.160113  0.042347
       Thur  0.160298  0.038774
Yes    Fri   0.174783  0.051293
       Sat   0.147906  0.061375
       Sun   0.187250  0.154134
       Thur  0.163863  0.039389

In [54]:
# 在DataFrame，可以指定应用的所有列上的函数列表或每一列上应用的不同函数
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
smoker day                                                          
No     Fri        4  0.151650  0.187735          4  18.420000  22.75
       Sat       45  0.158048  0.291990         45  19.661778  48.33
       Sun       57  0.160113  0.252672         57  20.506667  48.17
       Thur      45  0.160298  0.266312         45  17.113111  41.19
Yes    Fri       15  0.174783  0.263480         15  16.813333  40.17
       Sat       42  0.147906  0.325733         42  21.276667  50.81
       Sun       19  0.187250  0.710345         19  24.120000  45.35
       Thur      17  0.163863  0.241255         17  19.190588  43.11

In [55]:
result['tip_pct']

count      mean       max
smoker day                            
No     Fri       4  0.151650  0.187735
       Sat      45  0.158048  0.291990
       Sun      57  0.160113  0.252672
       Thur     45  0.160298  0.266312
Yes    Fri      15  0.174783  0.263480
       Sat      42  0.147906  0.325733
       Sun      19  0.187250  0.710345
       Thur     17  0.163863  0.241255

In [56]:
# 也可以传递(name, function)
ftuples = [('A', 'mean'), ('B', 'max')]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct           total_bill       
                    A         B          A      B
smoker day                                       
No     Fri   0.151650  0.187735  18.420000  22.75
       Sat   0.158048  0.291990  19.661778  48.33
       Sun   0.160113  0.252672  20.506667  48.17
       Thur  0.160298  0.266312  17.113111  41.19
Yes    Fri   0.174783  0.263480  16.813333  40.17
       Sat   0.147906  0.325733  21.276667  50.81
       Sun   0.187250  0.710345  24.120000  45.35
       Thur  0.163863  0.241255  19.190588  43.11

In [57]:
# 在不同列上应用不同的函数
grouped.agg({'tip': np.max, 'size': sum})

tip  size
smoker day              
No     Fri    3.50     9
       Sat    9.00   115
       Sun    6.00   167
       Thur   6.70   112
Yes    Fri    4.73    31
       Sat   10.00   104
       Sun    6.50    49
       Thur   5.00    40

In [59]:
grouped.agg({'tip': ['min', 'max', 'mean', 'std'], 'size': ['sum', 'count']})

tip                            size      
              min    max      mean       std  sum count
smoker day                                             
No     Fri   1.50   3.50  2.812500  0.898494    9     4
       Sat   1.00   9.00  3.102889  1.642088  115    45
       Sun   1.01   6.00  3.167895  1.224785  167    57
       Thur  1.25   6.70  2.673778  1.282964  112    45
Yes    Fri   1.00   4.73  2.714000  1.077668   31    15
       Sat   1.00  10.00  2.875476  1.630580  104    42
       Sun   1.50   6.50  3.516842  1.261151   49    19
       Thur  2.00   5.00  3.030000  1.113491   40    17

In [61]:
# 返回不含索引的聚合数据
no_index = tips.groupby(['day', 'smoker'], as_index=False).mean()
no_index

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [62]:
type(no_index)

pandas.core.frame.DataFrame